In [1]:
#Snowpark lib
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

#ConfigParser to read ini file
import configparser

config = configparser.ConfigParser()
config.read("/notebooks/notebooks/credentials.ini")

connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [23]:
# df = session.table("MEMBER_FUNDS_ENRICHED_DETAILS").to_pandas()
df = pd.read_csv("/data/funds.csv")

In [24]:
# df.to_csv("/data/funds.csv", index=False)

In [25]:
df.head()

,MEMBER_ID,MEMBER_NAME,MEMBER_EMPLOYMENT,MEMBER_GENDER,MEMBER_CITY_TOWN,MEMBER_STATE,MEMBER_CONTACT_VERIFIED,FUND_ID,CHURN_REASON,LATITUDE,...,MEMBER_AGE,INVESTMENT_AGE_GROUP,RETIREMENT_AGE,RETIREMENT_AGE_GROUP,TOTAL_FUNDS_INVESTED,NO_OF_CHURN,MEMBER_DOB_DT,ALLOCATION_DT,CHURN_DT,RETIREMENT_DT
0,MID000001,Vallie Bachman,The University of Sydney,Female,Hobart,Tasmania,N,FID000106,NOT APPLICABLE,-42.8806,...,44,Middle-aged,21,MORE_THAN_10_YEAR,1,NaN,1980-01-08,2004-11-18,9999-12-31,2045-01-08
1,MID000028,Jani Herritt,Murdoch University,Female,Sydney,New South Wales,Y,FID000137,NOT APPLICABLE,-33.8678,...,52,Senior,13,MORE_THAN_10_YEAR,1,NaN,1972-02-27,2001-07-08,9999-12-31,2037-02-27
2,MID000044,Jani Rulapaugh,Torrens University Australia,Male,Melbourne,Victoria,Y,FID000050,NOT APPLICABLE,-37.8142,...,43,Middle-aged,22,MORE_THAN_10_YEAR,1,NaN,1981-02-17,2002-12-10,9999-12-31,2046-02-17
3,MID000089,Alease Hollack,University of New England,Male,Darwin,Northern Territory,N,FID000186,NOT APPLICABLE,-12.4381,...,43,Middle-aged,22,MORE_THAN_10_YEAR,1,NaN,1981-12-19,2002-08-13,9999-12-31,2046-12-19
4,MID000098,Devorah Whobrey,The University of Notre Dame Australia,Female,Darwin,Northern Territory,Y,FID000184,NOT APPLICABLE,-12.4381,...,59,Senior,6,IN_NEXT_5_YEAR,1,NaN,1965-12-07,2005-08-05,9999-12-31,2030-12-07


In [26]:
df.isna().sum() / df.shape[0]

MEMBER_ID                                           0.000000
MEMBER_NAME                                         0.000000
MEMBER_EMPLOYMENT                                   0.000000
MEMBER_GENDER                                       0.000000
MEMBER_CITY_TOWN                                    0.000000
MEMBER_STATE                                        0.000000
MEMBER_CONTACT_VERIFIED                             0.000000
FUND_ID                                             0.000000
CHURN_REASON                                        0.000000
LATITUDE                                            0.000000
LONGITUDE                                           0.000000
COUNTRY                                             0.000000
COUNTRYCODE                                         0.000000
CAPITAL                                             0.000000
POPULATION                                          0.000000
FUND_TOTAL_ASSETS                                   0.000000
FUND_RETURN_TARGET_PERCE

NO_OF_CHURN column has almost 50% missing value, it wont be wise to impute these many values so will just drop it.                                     

In [27]:
df.columns

Index(['MEMBER_ID', 'MEMBER_NAME', 'MEMBER_EMPLOYMENT', 'MEMBER_GENDER',
       'MEMBER_CITY_TOWN', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED',
       'FUND_ID', 'CHURN_REASON', 'LATITUDE', 'LONGITUDE', 'COUNTRY',
       'COUNTRYCODE', 'CAPITAL', 'POPULATION', 'FUND_TOTAL_ASSETS',
       'FUND_RETURN_TARGET_PERCENTAGE', 'INVESTMENT_RISK_LEVEL',
       'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION',
       'FIXED_INCOME_BENCHMARK_ALLOCATION',
       'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION',
       'INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
       'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION',
       'PROPERTY_BENCHMARK_ALLOCATION', 'INFRA_BENCHMARK_ALLOCATION',
       'COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION',
       'FUND_RISK_LEVEL', 'FUND_RISK_CATEGORY',
       'NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS',
       'YEAR_5_RETURNS', 'YEAR_7_RETURNS', 'YEAR_10_RETURNS', 'SUPER_FEES',
       'PEN

In [32]:
data = df[["MEMBER_EMPLOYMENT",'MEMBER_GENDER','MEMBER_CITY_TOWN', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','COUNTRYCODE',
          'FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE', 'INVESTMENT_RISK_LEVEL',
       'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
       'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
       'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
           'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
           'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
           'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES', 'CHURN_FLAG', 'MEMBER_AGE',
           'INVESTMENT_AGE_GROUP',
           'RETIREMENT_AGE', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED','CHURN_FLAG']].copy()

In [34]:
data["CHURN_FLAG"] = data["CHURN_FLAG"].apply(lambda x: 1 if x=="Y" else 0)
# data

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [35]:
data["CHURN_FLAG"].unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [20]:
df.groupby(["MEMBER_EMPLOYMENT","CHURN_FLAG"])[["CHURN_FLAG"]].count()

CHURN_FLAG
MEMBER_EMPLOYMENT                 CHURN_FLAG            
Australian Catholic University    N                 6627
                                  Y                 2465
Australian National University    N                 6689
                                  Y                 2560
Avondale University               N                 6490
...                                                  ...
University of Southern Queensland Y                 2584
University of Technology Sydney   N                 6449
                                  Y                 2475
Victoria University               N                 6607
                                  Y                 2551

[74 rows x 1 columns]